# The Price is Right

Today we build a more complex solution for estimating prices of goods.

1. This notebook: create a RAG database with our 400,000 training data
2. Day 2.1 notebook: visualize in 2D
3. Day 2.2 notebook: visualize in 3D
4. Day 2.3 notebook: build and test a RAG pipeline with GPT-4o-mini
5. Day 2.4 notebook: (a) bring back our Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

Phew! That's a lot to get through in one day!

## PLEASE NOTE:

We already have a very powerful product estimator with our proprietary, fine-tuned LLM. Most people would be very satisfied with that! The main reason we're adding these extra steps is to deepen your expertise with RAG and with Agentic workflows.


In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from jupyter_notebook_ai_clone.llm_udemy.llm_engineering.week6.items import Item 
#from loaders import ItemLoader
from jupyter_notebook_ai_clone.llm_udemy.llm_engineering.week6.loaders import ItemLoader
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# environment

load_dotenv()
#os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
#os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


## Back to the pkl files

Much as we enjoyed data curating in week 6, we probably don't want to go through that whole process again!

Let's reuse the pkl files we created then. Either copy the files `train.pkl` and `test.pkl` from the Week 6 folder into this Week 8 folder, or you can also download them from here:

https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW?usp=drive_link

In [4]:
# With train.pkl in this folder, you can run this:

with open('train_small.pkl', 'rb') as file:
    train = pickle.load(file)

In [5]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nGOLDEN ICEPURE Replacement for Kenmore 6, WF293, Refrigerator Water Filter 3 PACK\n💦𝐂𝐞𝐫𝐭𝐢𝐟𝐢𝐜𝐚𝐭𝐢𝐨𝐧 - The GOLDEN ICEPURE Refrigerator water filter is certified by the NSF/ANSI 42 and meets the requirements of NSF/ANSI/CAN 372. We guarantee the quality, health, and environmental friendliness of our products and services. 💦𝐏𝐞𝐫𝐟𝐨𝐫𝐦𝐚𝐧𝐜𝐞 - Using multi-stage filtration technology and high-grade coconut shell carbon\n\nPrice is $23.00'

# Now create a Chroma Datastore

In Week 5, we created a Chroma datastore with 123 documents representing chunks of objects from our fictional company Insurellm.

Now we will create a Chroma datastore with 400,000 products from our training dataset! It's getting real!

Note that we won't be using LangChain, but the API is very straightforward and consistent with before.

Special note: if Chroma crashes and you're a Windows user, you should try rolling back to an earlier version of the Chroma library with:  
`!pip install chromadb==0.5.0`  
With many thanks to student Kelly Z. for finding this out and pointing to the GitHub issue [here](https://github.com/chroma-core/chroma/issues/2513). 

In [6]:
client = chromadb.PersistentClient(path=DB)

In [7]:
#client.delete_collection(name="products")

In [9]:
# Check if the collection exists and delete it if it does
collection_name = "products"
existing_collection_names = client.list_collections()  # This directly returns names now
if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

# delete collection  
#client.delete_collection(collection_name)

Deleted existing collection: products


# Introducing the SentenceTransfomer

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to a 384 dimensional dense vector space and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

Last time we used OpenAI embeddings to produce vector embeddings. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG


In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]

In [13]:
vector

array([-9.46715996e-02,  4.27619927e-02,  5.51620461e-02, -5.10995043e-04,
        1.16203064e-02, -6.80130571e-02,  2.76406296e-02,  6.06974177e-02,
        2.88530905e-02, -1.74128544e-02, -4.94346432e-02,  2.30992641e-02,
       -1.28614204e-02, -4.31402475e-02,  2.17509363e-02,  4.26548757e-02,
        5.10500111e-02, -7.79727772e-02, -1.23247221e-01,  3.67456004e-02,
        4.54122620e-03,  9.47938487e-02, -5.53098917e-02,  1.70641635e-02,
       -2.92872861e-02, -4.47124429e-02,  2.06784345e-02,  6.39320239e-02,
        2.27427911e-02,  4.87789996e-02, -2.33501615e-03,  4.72859144e-02,
       -2.86258757e-02,  2.30624750e-02,  2.45130155e-02,  3.95681709e-02,
       -4.33176644e-02, -1.02316692e-01,  2.79873190e-03,  2.39303671e-02,
        1.61556490e-02, -8.99083540e-03,  2.07255818e-02,  6.40123338e-02,
        6.89179599e-02, -6.98361397e-02,  2.89760833e-03, -8.10990036e-02,
        1.71122737e-02,  2.50654924e-03, -1.06529079e-01, -4.87733372e-02,
       -1.67762246e-02, -

In [14]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [15]:
description(train[0])

'GOLDEN ICEPURE Replacement for Kenmore 6, WF293, Refrigerator Water Filter 3 PACK\n💦𝐂𝐞𝐫𝐭𝐢𝐟𝐢𝐜𝐚𝐭𝐢𝐨𝐧 - The GOLDEN ICEPURE Refrigerator water filter is certified by the NSF/ANSI 42 and meets the requirements of NSF/ANSI/CAN 372. We guarantee the quality, health, and environmental friendliness of our products and services. 💦𝐏𝐞𝐫𝐟𝐨𝐫𝐦𝐚𝐧𝐜𝐞 - Using multi-stage filtration technology and high-grade coconut shell carbon'

In [16]:
for i in tqdm(range(0, len(train), 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+1000)]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|██████████| 20/20 [14:59<00:00, 45.00s/it]
